In [6]:
"""Convert Olympus's proprietary MATL data into a format readable my ImageJ."""

from tkinter import filedialog
import os
import tkinter as tk
import xml.etree.ElementTree as ET
from IPython.display import clear_output
%gui tk

root = tk.Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

def processMATL(path):
    # Save processed TileConfig file(s) in folder(s) with MATL file.
    if type(path) != str:
        raise TypeError("Input must be str.")
        
    omppath = path + "/matl.omp2info"
    if not os.path.isfile(omppath): # If MATL file does not exist, skip.
        return False
    
    # Import XML
    tree = ET.parse(omppath)
    root = tree.getroot()

    grouplist = list() # Get group number first
    for group_i in range(len(root)):
        if 'objectId' in root[group_i].attrib.keys():
            grouplist.append(group_i)

    dictlist = list()
    z = float(0)
    for group in grouplist:
        dictimg = dict()
        for img_i in range(len(root[group])): # Images
            if 'id' in root[group][img_i].attrib.keys():
                check = 0
                for prop in root[group][img_i]: # Properties
                    if prop.tag.endswith('image'):
                        name = prop.text
                        check += 1
                    if prop.tag.endswith('xIndex'):
                        x = size * int(prop.text)
                        check += 1
                    if prop.tag.endswith('yIndex'):
                        y = size * int(prop.text)
                        check += 1
                if check == 3: # Must have all 3 attributes before adding
                    dictimg[name[:-4]+".tif"] = (x,y,z) # Convert oir to tif
        dictlist.append(dictimg)

    spacer = '; ; '
    for num in range(len(dictlist)):
        with open(path + 'TileConfiguration' + str(num+1) + '.txt', 'w') as f: # Tuple to str
            f.write('dim=3\n')
            for key, value in dictlist[num].items():
                f.write(key + spacer + ''.join(str(value)) + '\n')
    return True

size = float(input("Image size? ")) * 0.9 # 20% overlap
directory = filedialog.askdirectory(title="Choose big folder directory")

did_something = False
for root, dirs, files in os.walk(directory):
    for i in dirs:
        if processMATL(directory + "/" + i + "/"):
            did_something = True

clear_output(wait=True)
print("Done!") if did_something else print("Didn't run anything. Check folder?")

Done!
